# Setting up a containerized EC2 instance

S3 upload options:
- Boto3:
    - http://stackabuse.com/example-upload-a-file-to-aws-s3/
    - http://boto3.readthedocs.io/en/latest/guide/migration.html#installation-configuration
    - https://gist.github.com/freewayz/1fbd00928058c3d682a0e25367cc8ea4
- Amazon CLI: 
    - https://docs.aws.amazon.com/cli/latest/userguide/using-s3-commands.html
    - https://aws.amazon.com/getting-started/tutorials/backup-to-s3-cli/
    
Both are available to Rackham,

Boto3:
```
module load python && virtualenv venv && source venv/bin/activate && pip
install boto3
# Or by running "pip install -y boto3 --user"
```

CLI: awscli/1.11.140

## Console test

https://aws.amazon.com/getting-started/tutorials/backup-to-s3-cli/

```
$ sudo apt install awscli
$ aws configure
AWS Access Key ID [None]: 
AWS Secret Access Key [None]:
(also used eu-central-1 for region, and json as format)
```

The above commang needs SSL certificates. To generate the aws keys:

https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/set-up-ami-tools.html?icmpid=docs_iam_console#ami-tools-managing-certs

```
$ openssl genrsa 2048 > aws-private.pem
$ openssl req -new -x509 -nodes -sha256 -days 365 -key aws-private.pem -outform PEM -out aws-certificate.pem

if in dire need for security use:
$ sudo apt-get install xclip
$ xclip -sel clip < ~/.ssh/aws-private.pem
```

```
aws s3 mb s3://my-first-backup-bucket
upload:
aws s3 cp “C:\users\my first backup.bak” s3://my-first-backup-bucket/
download:
aws s3 cp s3://my-first-backup-bucket/my-first-backup.bak ./
delete:
aws s3 rm s3://my-first-backup-bucket/my-first-backup.bak
```

anyway, on rackham tests work!



## Boto3 test

- https://boto3.readthedocs.io/en/latest/guide/s3-example-creating-buckets.html
- http://boto3.readthedocs.io/en/latest/reference/services/s3.html

conda install -c anaconda boto3


In [7]:
import boto3

s3 = boto3.client('s3')

s3.create_bucket(Bucket='jo8a7fn8sfn8', CreateBucketConfiguration={'LocationConstraint': 'eu-central-1'})

{'Location': 'http://jo8a7fn8sfn8.s3.amazonaws.com/',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Wed, 02 May 2018 14:29:21 GMT',
   'location': 'http://jo8a7fn8sfn8.s3.amazonaws.com/',
   'server': 'AmazonS3',
   'x-amz-id-2': 'Jz2XiyGad4yBOiFyBvd4Z7uXvHqaP2+B+3wnFRotaRICsD25+9pDJ3Vc7cqGaJVzPPDQjeZtDFg=',
   'x-amz-request-id': '588D5CD6FB259F3E'},
  'HTTPStatusCode': 200,
  'HostId': 'Jz2XiyGad4yBOiFyBvd4Z7uXvHqaP2+B+3wnFRotaRICsD25+9pDJ3Vc7cqGaJVzPPDQjeZtDFg=',
  'RequestId': '588D5CD6FB259F3E',
  'RetryAttempts': 0}}

In [9]:
import boto3

# Create an S3 client
s3 = boto3.client('s3')

# Call S3 to list current buckets
response = s3.list_buckets()

# Get a list of all bucket names from the response
buckets = [bucket['Name'] for bucket in response['Buckets']]

# Print out the bucket list
print("Bucket List: %s" % buckets)

Bucket List: ['jo8a7fn8sfn8']


In [10]:
import boto3

# Create an S3 client
s3 = boto3.client('s3')

filename = '/media/sergiu/workpc/data/work/andersson/test/temp/crass.crispr'
bucket_name = 'jo8a7fn8sfn8'

# Uploads the given file using a managed uploader, which will split up large
# files automatically and upload parts in parallel.
s3.upload_file(filename, bucket_name, filename)

# or
s3.Object('mybucket', 'hello.txt').put(Body=open('/tmp/hello.txt', 'rb'))


In [13]:
# https://boto3.readthedocs.io/en/latest/guide/migrations3.html#deleting-a-bucket
import boto3
import botocore

s3 = boto3.resource('s3')
bucket = s3.Bucket('jo8a7fn8sfn8')

exists = True
try:
    s3.meta.client.head_bucket(Bucket='jo8a7fn8sfn8')
except botocore.exceptions.ClientError as e:
    # If a client error is thrown, then check that it was a 404 error.
    # If it was a 404 error, then the bucket does not exist.
    error_code = int(e.response['Error']['Code'])
    if error_code == 404:
        exists = False

if exists==True:
    for key in bucket.objects.all():
        key.delete()
    bucket.delete()

## Install Docker CE on Ubuntu

https://docs.docker.com/install/linux/docker-ce/ubuntu/

```
$ sudo apt-get update
# allow apt to use https repos
$ sudo apt-get install \
    apt-transport-https \
    ca-certificates \
    curl \
    software-properties-common
# add the repo key
$ curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

$ sudo add-apt-repository \
   "deb [arch=amd64] https://download.docker.com/linux/ubuntu \
   $(lsb_release -cs) \
   stable"
$ sudo apt-get update
$ sudo apt-get install docker-ce
```

Now following this to link the image directories to my hdd:
https://forums.docker.com/t/how-do-i-change-the-docker-image-installation-directory/1169

```
sudo nano /etc/default/docker
DOCKER_OPTS="--dns 8.8.8.8 --dns 8.8.4.4 -g /media/sergiu/lappie/docker"
service docker restart
```